### Merkle Tree


In [1]:
from hashlib import sha256

class MerkleTree:
    def __init__(self, leaves: list[int]):
        assert len(leaves) & (len(leaves) - 1) == 0, "Leaves must be a power of 2"
        self.leaves = leaves
        self.tree = []
        self.root = self._build_tree()

    def get_path(self, leaf_index: int) -> list[int]:
        path = []
        layer_index = 0
        path.append(self.tree[layer_index][leaf_index])
        while layer_index < len(self.tree) - 1:
            path.append(self.tree[layer_index][leaf_index + 1 if leaf_index % 2 else leaf_index - 1])
            layer_index += 1
            leaf_index = leaf_index // 2
        return path


    def _build_tree(self) -> list[int]:
        curr_layer = [sha256(str(leaf).encode()).hexdigest() for leaf in self.leaves]
        self.tree.append(curr_layer)
        while len(curr_layer) > 1:
            next_layer = self._build_layer(curr_layer)
            self.tree.append(next_layer)
            curr_layer = next_layer
        return curr_layer[0]
        

    def _build_layer(self, layer: list[int]) -> list[int]:
        new_layer = []
        for i in range(0, len(layer), 2):
            new_layer.append(sha256((layer[i]+ layer[i + 1]).encode()).hexdigest())
        return new_layer


In [2]:
from pprint import pprint

m = MerkleTree([1,2,3,4,5,6,7,8])
pprint([[b[:4] for b in a] for a in m.tree])
print(m.root)
pprint([a[:4] for a  in m.get_path(0)])

[['6b86', 'd473', '4e07', '4b22', 'ef2d', 'e7f6', '7902', '2c62'],
 ['33b6', '1365', '4358', 'ada1'],
 ['85df', 'e0e2'],
 ['c274']]
c27450cd3fd4df029145f3437ae9c381e0ae55e8400de06cb973005b36d7b222
['6b86', '2c62', 'ada1', 'e0e2']


### Hash Chain

In [3]:
import random

class HashChain:
    def __init__(self, seed: str):
        self.chain = [seed]
        self.random_generator = random.Random(seed)

    def get_next_random(self):
        nonce = self.random_generator.randbytes(4)
        self.chain.append(sha256(self.chain[-1].encode() + nonce).hexdigest())
        return int(self.chain[-1], 16)

    def get_all_hashes(self):
        return self.chain

### Channel

In [4]:
from queue import Queue

class Channel:
    def __init__(self, F):
        """Initialize the channel with an optional random seed."""
        self.messages = Queue()
        self.F = F

    def send(self, message: dict[str, int | str]):
        """Send (store) a message in the channel."""
        self.messages.put(message)

    def receive(self):
        """Receive (retrieve) the last message sent."""
        return self.messages.get()
    
    def get_all_messages(self):
        """Retrieve all messages exchanged."""
        return list(self.messages.queue)
    
    def _get_hash(self):
        return sha256("".join(map(lambda x: x["data"], self.messages.queue)).encode()).hexdigest()
    
    def receive_random_field_element(self, title: str = "Random Field Element"):
        """Generate a random f"""
        random_field_element = self.F(int(self._get_hash(), 16))
        self.messages.put({
            "title": title,
            "data": str(random_field_element)
        })
        return random_field_element

### Fibonacci function

In [5]:
def fibonacci(a=1, size=1001) -> list[int]:
    fib_list = [1, a]
    for _ in range(2, size):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list 

In [6]:
print(fibonacci())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777836

### Polynomial f(x)

In [7]:
P: int =3221225473
F = GF(P)
N = 8192

g = F(1734477367)
gamma = g ** 8

g_group = [g ** i for i in range(N)]
gamma_group = [gamma ** i for i in range(1001)]

w = F.multiplicative_generator()

wg = [w * gi for gi in g_group]

fibonacci_seq = fibonacci()

R.<x> = PolynomialRing(F)

f_x = R.lagrange_polynomial(zip(gamma_group, fibonacci_seq))

### Bit Reverse Order

In [8]:
def bit_reverse(n, width):
    rev = 0  # This will store the reversed bits
    for i in range(width):
        rev = (rev << 1) | (n & 1)  # Shift left and take LSB of n
        n >>= 1  # Shift n to the right
    return rev

def bit_reverse_permutation(N):
    """Generate bit-reversed order for N elements."""
    width = int(math.log(N, 2))
    return [bit_reverse(i, width) for i in range(N)]

### First Commit

In [9]:
f_x_on_wg = [f_x(x) for x in wg]

bit_reverse_order = bit_reverse_permutation(N)
f_x_on_wg_merkle_tree_order = [f_x_on_wg[i] for i in bit_reverse_order]
merkle_f_wg = MerkleTree(f_x_on_wg_merkle_tree_order)
original_commit =  merkle_f_wg.root

channel = Channel(F)
channel.send({'title': 'original_commit', 'data': original_commit})


### Polynomial h(x)

In [ ]:
d_x = 1
for gamma_i in gamma_group[:-2]:
    d_x = d_x * (x - gamma_i)


F_x = f_x(x) + f_x(gamma * x) - f_x((gamma**2) * x)
F_x_on_wg = [F_x(x) for x in wg]

d_x_on_wg = [d_x(x) for x in wg]


# h_x = F_x // d_x
h_x_on_wg = [F_x_on_wg[i] / d_x_on_wg[i] for i in range(len(wg))]


[396904994, 1340430823, 1745936784, 838585142, 1710346573, 949072581, 3091599597, 1695969674, 1616325697, 2998753023, 1760329759, 2597124584, 2901474875, 1900844809, 1051922882, 17322172, 1298709708, 2243905963, 1802402394, 477639933, 3054201994, 2575710863, 2956545242, 2000883320, 1068502394, 1352327987, 154844308, 1110427025, 2943845571, 719472086, 1384517229, 782588481, 2300285139, 2208963363, 169945988, 427010067, 1941252356, 1517934390, 1575644531, 2061803909, 2236147504, 253888239, 2952796460, 916084032, 1055095844, 1707037138, 695999592, 2592510126, 580501896, 705573855, 487122652, 2801335192, 3047240472, 157383659, 1988876275, 2405528526, 387578945, 1336950022, 1267392982, 2415770419, 2704309461, 535738560, 357974745, 1887755551, 210521644, 1144506767, 1571360585, 3151053278, 1961054360, 1990982182, 125208966, 1030391903, 2817951101, 1101928997, 794031688, 1047544864, 1226762068, 2872063819, 1998819623, 1284386547, 2644137550, 1687291739, 2264693678, 602796676, 1821978517, 1157

### Boundary Constraints

In [17]:
Y = fibonacci_seq[-1]

# t_0_x = (f_x(x) - 1) / (x - gamma ** 0)

# t_1_x = (f_x(x) - Y) / (x - gamma ** 1000)



t_0_on_wg = [(f_x(i) - 1) / (i - gamma ** 0) for i in wg]
t_1_on_wg = [(f_x(i) - Y) / (i - gamma ** 1000) for i in wg]

### FRI - Composition Polynomial

In [37]:
channel = Channel(F) # TODO: Delete
channel.send({'title': 'original_commit', 'data': original_commit}) # TODO: Delete
beta_0 = channel.receive_random_field_element()
beta_1 = channel.receive_random_field_element()
beta_2 = channel.receive_random_field_element()

print(beta_0, beta_1, beta_2)

# cp_0_x = beta_0 * h_x + beta_1 * t_0_x + beta_2 * t_1_x

# print(cp_0_x)

cp_0_on_wg = [ beta_0 * h_x_on_wg[i] + beta_1 * t_0_on_wg[i] + beta_2 * t_1_on_wg[i] for i in range(len(wg))]

reverse_ordered_cp = [cp_0_on_wg[i] for i in bit_reverse_order]

# merkle_cp_0 = MerkleTree(cp_0_on_wg_reverse)
# channel.send({'title': 'commit for degree 0', 'data': merkle_cp_0.root})




cps = []
cp_on_wg_prev = reverse_ordered_cp
random_x = channel.receive_random_field_element()

for _ in range(13):
    # reverse_ordered_cp = [cp_on_wg_prev[i] for i in bit_reverse_order]
    cp_on_wg_next = [((cp_on_wg_prev[i]+ cp_on_wg_prev[i + 1]) // 2) + \
                     random_x * (cp_on_wg_prev[i]+ cp_on_wg_prev[i + 1]) // (2 * cp_on_wg_prev[i]) for i in range(len(cp_on_wg_prev))[::2]]
    cps.append(cp_on_wg_prev)
    cp_on_wg_prev = cp_on_wg_next
    print(cp_on_wg_next[:4])
    channel.send({
        "title": "commit for layer {}".format(len(cps) - 1),
        "data": MerkleTree(cp_on_wg_next).root,
    })




# cp_prev = R(cp_0_x)
# cps = []
# print("channel before FRI", channel.get_all_messages())
# while cp_prev.degree() > 0:
#     cp_prev_on_wg = [cp_prev(i) for i in wg]

#     merk = MerkleTree([cp_prev_on_wg[i] for i in bit_reverse_order])
#     commit = merk.root
#     channel.send({'title': f'commit for degree {cp_prev.degree()}', 'data': commit})
    
#     random_x = channel.receive_random_field_element()
#     cp_next = (cp_prev(x) + cp_prev(-x)) // 2 + random_x * (cp_prev(x) - cp_prev(-x)) // (2 * x)
#     cp_next = sum(coef * x^(exp // 2) for exp, coef in R(cp_next).dict().items())
#     cps.append(cp_prev)
#     cp_prev = cp_next

# channel.send({'title': 'final_commit', 'data': cp_prev})

# def verify_merkle_path(merkle_root: str, merkle_path: list[str]):
#     root = merkle_path[0]
#     for sibling in merkle_path[1:]:
#         root = sha256((root + sibling).encode()).hexdigest()
#     return root == merkle_root




592641122 174315803 2915487417
[1383912501, 1150703630, 762456835, 2571800605]
[1422861046, 1737161958, 2913165180, 525375524]
[1520379842, 1456253191, 1242666150, 1543792834]
[1280914562, 66567990, 2415378738, 3079573272]
[647725633, 1118568286, 2651926383, 293364002]
[3204640505, 899609283, 1942333479, 959074850]
[2548888869, 2089066443, 2589699105, 490161870]
[2627867309, 2976209360, 1870121228, 2274556232]
[697992216, 2976902037, 2140589701, 815846025]
[170579957, 2511138306, 631935089, 2509547240]
[1014554451, 1663812799, 940913963, 891556192]
[520090075, 1566731955]
[2302396327]


## Decommit Phase

In [ ]:
from random import randint, seed
seed(42069)

rand_idx = randint(0, len(wg))
rand_x = wg[rand_idx]
f_x_at_idx = f_x(rand_x)
f_x_at_g_idx = f_x(gamma * rand_x)
f_x_at_g2_idx = f_x(gamma**2 * rand_x)

channel.send({'title': 'decommit phase 0 ', 'data': {'result': f_x_at_idx, 'path': merkle_f_wg.get_path(rand_idx)}})

for i in range(len(cps)):
    cp = cps[i]
    cp_at_idx = cp(rand_x)
    cp_at_neg_idx = cp(-rand_idx)




8192